In [5]:
import matplotlib.pyplot as plt
import numpy as np 
import random
from matplotlib import style
style_in_matplot=plt.style.available
style.use(str(style_in_matplot[5]))

In [2]:
def create_plots():
    xs=[]
    ys=[]
    for i in range(5):
        x=i
        y=random.randrange(10)
        xs.append(x)
        ys.append(y)
    return xs,ys


In [58]:

fig=plt.figure()
#First two integer ij gives a matrix and last element show the index
#e.g 32 gives 6 so whole plot is divided into 6 slot 323 means 3 slot out of 6
#32 means divide the whole plain into 3 horizontal plot and 
#then dividing each plot into two
ax1=fig.add_subplot(322)
x,y=create_plots()
ax1.plot(x,y,'-',label="axis1")
plt.legend()
ax3=fig.add_subplot(312)
x,y=create_plots()
ax3.plot(x,y,label="x3")
plt.legend()
ax2=fig.add_subplot(339)
x,y=create_plots()
ax2.plot(x,y,label="x2")
plt.legend()
plt.show()

In [70]:
fig=plt.figure()
ax1=plt.subplot2grid((6,1),(0,0),rowspan=2,colspan=1)
x,y=create_plots()
ax1.plot(x,y,'-',label="axis1")
plt.legend()
ax2=plt.subplot2grid((6,2),(2,1),rowspan=1,colspan=1)
x,y=create_plots()
ax2.plot(x,y,'-',label="axis2")
plt.legend()
ax2=plt.subplot2grid((6,1),(4,0),rowspan=2,colspan=1)
x,y=create_plots()
ax2.plot(x,y,'-',label="axis3")
plt.legend()
plt.show()

In [6]:
import urllib
def graph_data(stock,time):
    stock_price_url = 'http://chartapi.finance.yahoo.com/instrument/1.0/'+stock+'/chartdata;type=quote;range='+time+'/csv'

    source_code = urllib.urlopen(stock_price_url).read().decode()

    stock_data = []
    split_source = source_code.split('\n')

    for line in split_source:
        split_line = line.split(',')
        if len(split_line) == 6:
            if 'values' not in line:
                stock_data.append(line)
    
    return stock_data

In [25]:

import matplotlib.dates as mdates
import time
import datetime as dt 
company="GOOG"
time='10y'
stock_data=graph_data(company,time)
date, closep, highp, lowp, openp, volume = np.loadtxt(stock_data,
                                                      delimiter=',',unpack=True)
def string_to_matplotdates(date_list,fmt):
    dates=[]
    for d in date_list:
        dates.append(dt.datetime.strptime(str(int(d)),'%Y%m%d'))
    dates = mdates.date2num(dates)  
    return dates
date=string_to_matplotdates(date,'%Y%m%d')

In [ ]:
def moving_average(values,window):
    weights=np.repeat(1.0,window)/window
    smas=np.convolve(values,weights,'valid')
    return smas
def high_minus_low(highs,lows):
    if type(highs)==list:
        return list(map(high_minus_low,a,b))
    else:
        return highs-lows

In [81]:
##please set data of 1m to get proper visual
import matplotlib.ticker as mticker
from matplotlib.finance import candlestick_ohlc
fig=plt.figure(facecolor="#30a2da")

#Subplot 1 defined
ax1=plt.subplot2grid((6,1),(0,0),rowspan=1,colspan=1)
plt.title(company)
plt.ylabel("H-L")


#Major function call for plotting in subplot 1

h_l=high_minus_low(highp,lowp)
ax1.plot_date(date,h_l,'-',label="High-Low")


#For better viewing purpose in subplot 1

ax1.yaxis.set_major_locator(mticker.MaxNLocator(nbins=3))#No. of pints in y axis
plt.setp(ax1.get_xticklabels(),visible=False)#dont show the x labels of graph 1

ax1.legend()
leg=ax1.legend(loc=9,ncol=1,prop={"size":10})#customising legend
leg.get_frame().set_alpha(0.4)


#Subplot 2 defined
ax2=plt.subplot2grid((6,1),(1,0),rowspan=4,colspan=1,sharex=ax1)#sharing x -axis with ax1
plt.ylabel("Price")
ax2v=ax2.twinx()#for purpose of showing different value in same chart ignoring y values or scaling as per previous x-values

#Major function call for plotting in subplot 2

x=0
y=len(date)
ohlc=[]
while x<y:
    append_me=date[x],openp[x],highp[x],lowp[x],closep[x],volume[x]
    ohlc.append(append_me)
    x+=1

candlestick_ohlc(ax2,ohlc,width=0.4,colorup='g',colordown='r')


#For better viewing purpose in subplot 2
bbox_props = dict(boxstyle="round,pad=0.1", fc="cyan", ec="b", lw=1,alpha=0.3)#More box http://matplotlib.org/users/annotations_guide.html
ax2.annotate("$"+str(closep[-1]),(date[-1],closep[-1]),#position pointing to
            xytext=(date[-1],closep[-1])#original position of text
             ,color="blue",bbox=bbox_props)
    

ax2.yaxis.set_major_locator(mticker.MaxNLocator(nbins=7,prune="upper") )
plt.setp(ax2.get_xticklabels(),visible=False)  


ax2v.fill_between(date,0,volume,facecolor="#0079a3",alpha=0.3)#showing volume in the same chart

ax2v.plot([],[],color="#0079a3",alpha=0.3,label="Volume")#for viewing pupose as we cannot label in fill
ax2v.axes.yaxis.set_ticklabels([])#for not showing y axis label in right side
ax2v.set_ylim(0,3*volume.max())#for making the volume spikes reasonably small
ax2v.grid(False)

ax2v.legend()
leg=ax2v.legend(loc=9,ncol=1,prop={"size":10})#customising legend
leg.get_frame().set_alpha(0.4)
ax2.grid(True)


#Subplot 3 defined
ax3=plt.subplot2grid((6,1),(5,0),rowspan=1,colspan=1,sharex=ax1)#sharing x -axis with ax1
plt.ylabel("Mov avg")

#Major function call for plotting in subplot 3

MA1=10
MA2=30
ma1=moving_average(closep,MA1)
ma2=moving_average(closep,MA2)
start=len(date[MA2-1:])

ax3.plot(date[-start:],ma1[-start:],linewidth=1,label=str(MA1)+"MA")
ax3.plot(date[-start:],ma2[-start:],linewidth=1,label=str(MA2)+"MA")
ax3.fill_between(date[-start:],ma2[-start:],ma1[-start:],where=(ma2[-start:]>ma1[-start:]),facecolor="red",
                edgecolor='r',alpha=0.5)
ax3.fill_between(date[-start:],ma2[-start:],ma1[-start:],where=(ma2[-start:]<ma1[-start:]),facecolor="green",
                edgecolor='g',alpha=0.5)
  
    
#For better viewing purpose in subplot 3

ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))#formating date
ax3.xaxis.set_major_locator(mticker.MaxNLocator(nbins=7))#no of pints in axis x
ax3.yaxis.set_major_locator(mticker.MaxNLocator(nbins=3,prune="upper"))#no of pints in axis y                        
plt.xlabel("Date")

for label in ax3.xaxis.get_ticklabels():
    label.set_rotation(45)
    
ax3.legend()
leg=ax3.legend(loc=9,ncol=2,prop={"size":10})#ncol=2 gives two column for two labels
leg.get_frame().set_alpha(0.4)


plt.subplots_adjust(left=0.09,bottom=0.20,right=0.94,top=0.90,wspace=0.2,hspace=0)

plt.show()
fig.savefig("google.png",facecolor='#30a2da')

In [77]:
print plt.__file__

C:\Users\RSB\Anaconda2\envs\dato-env\lib\site-packages\matplotlib\pyplot.pyc
